In [1]:
import numpy as np
import pymc3 as pm
import theano.tensor as tt

In [52]:
def stick_breaking(beta):
    portion_remaining = tt.concatenate([[1], tt.extra_ops.cumprod(beta)[:-1]])
    return beta * portion_remaining

In [53]:
n = 10
K = 10
with pm.Model() as model:
    # Params for Beta distribution over factor probabilities
    #a = pm.Gamma("a", 10, 1)
    a = pm.Gamma("a", 1, 1)
    beta = pm.Beta("beta", a, 1, shape = 10)
    p = stick_breaking(beta)
    #b = pm.Gamma("b", 10, 1)
    #p = pm.Beta("p", a/K, b*(K-1)/K, shape = 10)
    #p = pm.Beta("p", a, b, shape = 10)
    #p = tt.tile(p[None, :], (100, 1))
    # Bernoulli indicator matrix of which factors are "on" for every observation
    a1 = pm.Gamma('a1', alpha=tt.tile(p[None, :], (100, 1))/n, beta=1, shape=(100, 10))
    a2 = pm.Gamma("a2", alpha = (1-tt.tile(p[None, :], (100, 1)))/n, beta = 1, shape = (100, 10))
    Z = pm.Deterministic("Z", a1/(a1+a2))
    # Latent factors - 10 of them
    factors = pm.Normal("factors", mu = 0, sd = 1, shape = 10)
    # Observations are linear combinations of the factors
    equation = tt.sum(Z * tt.tile(factors[None, :], (100, 1)), axis = -1)
    sd = pm.HalfCauchy("sd", 0.5)
    obs = pm.Normal("obs", mu = equation, sd = sd, observed = data)

with model:
    tr = pm.sample(tune = 4000, draws = 2000, njobs = 3)

NameError: name 'data' is not defined

In [69]:
def stick_breaking(v):
    batch_ndims = len(v.shape) - 1
    cumprod_one_minus_v = np.exp(np.log1p(-v).cumsum(-1))
    one_v = np.pad(v, [[0, 0]] * batch_ndims + [[0, 1]], constant_values=1)
    c_one = np.pad(
        cumprod_one_minus_v, [[0, 0]] * batch_ndims + [[1, 0]], constant_values=1
    )
    return one_v * c_one

In [74]:
p = pm.Beta.dist(0.5, 0.5).random(size=10)

In [75]:
stick_breaking(p)

array([9.04451433e-01, 1.98691102e-02, 6.49683553e-04, 8.11862313e-06,
       1.77193557e-02, 1.04669427e-02, 3.30355434e-02, 7.37744465e-03,
       4.02766270e-03, 2.30944460e-03, 8.52609423e-05])

In [49]:
tt.tile(p[None, :], (100, 1)).eval()

array([[0.99828256, 0.24603622, 0.97772633, 0.91130333, 0.62828911,
        0.99623664, 0.4814024 , 0.30163583, 0.0446408 , 0.81320321],
       [0.99828256, 0.24603622, 0.97772633, 0.91130333, 0.62828911,
        0.99623664, 0.4814024 , 0.30163583, 0.0446408 , 0.81320321],
       [0.99828256, 0.24603622, 0.97772633, 0.91130333, 0.62828911,
        0.99623664, 0.4814024 , 0.30163583, 0.0446408 , 0.81320321],
       [0.99828256, 0.24603622, 0.97772633, 0.91130333, 0.62828911,
        0.99623664, 0.4814024 , 0.30163583, 0.0446408 , 0.81320321],
       [0.99828256, 0.24603622, 0.97772633, 0.91130333, 0.62828911,
        0.99623664, 0.4814024 , 0.30163583, 0.0446408 , 0.81320321],
       [0.99828256, 0.24603622, 0.97772633, 0.91130333, 0.62828911,
        0.99623664, 0.4814024 , 0.30163583, 0.0446408 , 0.81320321],
       [0.99828256, 0.24603622, 0.97772633, 0.91130333, 0.62828911,
        0.99623664, 0.4814024 , 0.30163583, 0.0446408 , 0.81320321],
       [0.99828256, 0.24603622, 0.9777263

In [35]:
tt.tile()

TypeError: tile() missing 2 required positional arguments: 'x' and 'reps'

In [28]:
tfp.distributions.ExpRelaxedOneHotCategorical(
    temperature, logits=None, probs=None, validate_args=False, allow_nan_stats=True,
    name='ExpRelaxedOneHotCategorical'
)
